In [1]:
import os
import time
import pandas as pd
from pyspark.sql.functions import mean

In [2]:
st=time.time()
directory_path = '/home/hadoop/pyspark/sokulee/'
f = open('sokulee.csv', 'w')
f.write('id,sleep,steps,wearing_time\n')
for n in range(1,100):
    name = 'A0' + str(n)
    directory_name = directory_path + name

    if os.path.exists(directory_name):
        df_sleep_temp = spark.read.json(directory_name + '/*_sleep.json')
        df_steps_temp = spark.read.json(directory_name + '/*_steps.json')
        df_heart_temp = spark.read.json(directory_name + '/*_heart.json')

        a = df_sleep_temp.select(mean(df_sleep_temp['summary']['totalTimeInBed']).alias('sleep'))
        b = df_steps_temp.select(mean(df_steps_temp['activities-steps'][0]['value']).alias('steps'))
        c = df_heart_temp.select(mean(df_heart_temp['activities-heart'][0]['value']['heartRateZones'][0]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][1]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][2]['minutes'] 
                    + df_heart_temp['activities-heart'][0]['value']['heartRateZones'][3]['minutes']).alias('time'))
        
        f.write(name + ',' + str(a.collect()[0][0]) + ',' + str(b.collect()[0][0]) + ',' + str(c.collect()[0][0]) + '\n')
        
f.close()
print(time.time()-st, 'sec')

292.107515335083 sec


In [3]:
df_sokulee = pd.read_csv('sokulee.csv')
df_sokulee.head()

,id,sleep,steps,wearing_time
0,A01,245.306122,7030.36,956.439024
1,A02,206.387755,6357.46,938.333333
2,A03,396.448980,7826.04,1157.320000
3,A04,358.224490,13340.86,1312.320000
4,A05,64.632653,4116.30,944.071429


In [4]:
import math
from operator import itemgetter
from scipy.spatial import distance

def distance_euclidean(a, b):
    return 1/(distance.euclidean(a,b)+1)

In [5]:
def nearest_neighbor_user(user, topN, simFunc):
    nn = {}
    interSectionU1 = []
    for r in df_sokulee.loc[df_sokulee['id'] == user].iterrows():
        interSectionU1.append(r[1][1])
        interSectionU1.append(r[1][2])
        interSectionU1.append(r[1][3])
    
    ## Brote Force Compute
    for uid in df_sokulee['id']:
        interSectionU2 = []
        
        if uid == user:
            continue
        
        for r in df_sokulee.loc[df_sokulee['id'] == uid].iterrows():
            interSectionU2.append(r[1][1])
            interSectionU2.append(r[1][2])
            interSectionU2.append(r[1][3])
            
        ## similarity function
        sim = simFunc(interSectionU1, interSectionU2)
        
        if math.isnan(sim) == False:
            nn[uid] = sim
            
    ## top N returned
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [6]:
sleep_min = df_sokulee['sleep'].min()
sleep_max = df_sokulee['sleep'].max()
steps_min = df_sokulee['steps'].min()
steps_max = df_sokulee['steps'].max()
wearing_time_min = df_sokulee['wearing_time'].min()
wearing_time_max = df_sokulee['wearing_time'].max()
print(sleep_min, sleep_max)
print(steps_min, steps_max)
print(wearing_time_min, wearing_time_max)

13.98 497.510204082
4116.3 21666.4
744.875 1402.24


In [7]:
df_sokulee['sleep'] = (df_sokulee['sleep'] - sleep_min) / (sleep_max - sleep_min)
df_sokulee['steps'] = (df_sokulee['steps'] - steps_min) / (steps_max - steps_min)
df_sokulee['wearing_time'] = (df_sokulee['wearing_time'] - wearing_time_min) / (wearing_time_max - wearing_time_min)
df_sokulee.head()

,id,sleep,steps,wearing_time
0,A01,0.478411,0.166042,0.321836
1,A02,0.397923,0.127701,0.294294
2,A03,0.790993,0.211380,0.627422
3,A04,0.711940,0.525613,0.863211
4,A05,0.104756,0.000000,0.303023


In [8]:
neighbor = nearest_neighbor_user('A08', 10, distance_euclidean)
for n in neighbor:
    print(n)

('A052', 0.989020311616692)
('A06', 0.9278684952864951)
('A068', 0.9272284561577543)
('A07', 0.9094979561621419)
('A017', 0.9064484165527119)
('A032', 0.8971041460397456)
('A04', 0.8894261981856548)
('A018', 0.8360769427114618)
('A083', 0.832476319672743)
('A019', 0.832418729285905)


In [9]:
neighbor = nearest_neighbor_user('A08', 1, distance_euclidean)
for n in neighbor:
    print(n)

('A052', 0.989020311616692)
